In [1]:
import csv
import IPython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import warnings

import tensorflow as tf
import tensorflow_probability as tfp

In [2]:
import urllib
import os
os.getcwd()

'/Users/yafeng/Documents/tfp'

In [3]:
# We'll use the following directory to store files we download as well as our
# preprocessed dataset.
CACHE_DIR = os.path.join(os.sep, '/projects/bbr/code_yc/tfp/test_data/tmp', 'radon')


def cache_or_download_file(cache_dir, url_base, filename):
    """Read a cached file or download it."""
    filepath = os.path.join(cache_dir, filename)
    if tf.gfile.Exists(filepath):
        return filepath
    if not tf.gfile.Exists(cache_dir):
        tf.gfile.MakeDirs(cache_dir)
    url = os.path.join(url_base, filename)
    print("Downloading {url} to {filepath}.".format(url=url, filepath=filepath))
    urllib.request.urlretrieve(url, filepath)
    return filepath


def download_radon_dataset(cache_dir=CACHE_DIR):
    """Download the radon dataset and read as Pandas dataframe."""
    url_base = 'http://www.stat.columbia.edu/~gelman/arm/examples/radon/'
    # Alternative source:
    # url_base = ('https://raw.githubusercontent.com/pymc-devs/uq_chapter/'
    #             'master/reference/data/')
    srrs2 = pd.read_csv('test_data/srrs2.csv')
    srrs2.rename(columns=str.strip, inplace=True)
    cty = pd.read_csv('test_data/cty.csv')
    cty.rename(columns=str.strip, inplace=True)
    return srrs2, cty


def preprocess_radon_dataset(srrs2, cty, state='MN'):
    """Preprocess radon dataset as done in "Bayesian Data Analysis" book."""
    srrs2 = srrs2[srrs2.state==state].copy()
    cty = cty[cty.st==state].copy()

    # We will now join datasets on Federal Information Processing Standards
    # (FIPS) id, ie, codes that link geographic units, counties and county
    # equivalents. http://jeffgill.org/Teaching/rpqm_9.pdf
    srrs2['fips'] = 1000 * srrs2.stfips + srrs2.cntyfips
    cty['fips'] = 1000 * cty.stfips + cty.ctfips

    df = srrs2.merge(cty[['fips', 'Uppm']], on='fips')
    df = df.drop_duplicates(subset='idnum')
    df = df.rename(index=str, columns={'Uppm': 'uranium_ppm'})

    # For any missing or invalid activity readings, we'll use a value of `0.1`.
    df['radon'] = df.activity.apply(lambda x: x if x > 0. else 0.1)

    # Remap categories to start from 0 and end at max(category).
    county_name = sorted(df.county.unique())
    df['county'] = df.county.astype(
        pd.api.types.CategoricalDtype(categories=county_name)).cat.codes
    county_name = map(str.strip, county_name)

    df['radon'] = df['radon'].apply(np.log)
    df['log_uranium_ppm'] = df['uranium_ppm'].apply(np.log) 
    df = df[['radon', 'floor', 'county', 'log_uranium_ppm']]

    return df, county_name

In [4]:
# radon, county_name = preprocess_radon_dataset(*download_radon_dataset())
radon = pd.read_csv('test_data/tmp/radon/radon.csv')

radon['radon'] = (np.round(np.exp(radon.radon),1)*10).astype(int)

In [5]:
radon.radon.values.std()

4.698429614875686e+18

In [6]:
CACHE_DIR

'/projects/bbr/code_yc/tfp/test_data/tmp/radon'

In [7]:
# with tf.gfile.Open(os.path.join(CACHE_DIR, 'radon.csv'), 'w') as f:
#     radon.to_csv(f, index=False)

In [8]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    tf.reset_default_graph()
    try:
        sess.close()
    except:
        pass
    sess = tf.InteractiveSession()

In [9]:
inv_scale_transform = lambda y: np.log(y)  # Not using TF here.
fwd_scale_transform = tf.exp

In [10]:
def _make_weights_prior(num_counties, dtype):
    """Returns a `len(log_uranium_ppm)` batch of univariate Normal."""
    raw_prior_scale = tf.get_variable(
        name='raw_prior_scale',
        initializer=np.array(inv_scale_transform(1.), dtype=dtype))
    return tfp.distributions.Independent(
        tfp.distributions.Normal(
            loc=tf.zeros(num_counties, dtype=dtype),
            scale=fwd_scale_transform(raw_prior_scale)),
        reinterpreted_batch_ndims=1)


make_weights_prior = tf.make_template(
    name_='make_weights_prior', func_=_make_weights_prior)

In [11]:
def _make_radon_likelihood(
    random_effect_weights, floor, county,log_county_uranium_ppm, init_radon_stddev
):
    raw_likelihood_scale = tf.get_variable(
        name='raw_likelihood_scale',
        initializer=np.array(
            inv_scale_transform(init_radon_stddev), dtype=dtype))
    fixed_effect_weights = tf.get_variable(
        name='fixed_effect_weights', initializer=np.array([0., 1.], dtype=dtype))
    fixed_effects = fixed_effect_weights[0] + fixed_effect_weights[1] * floor
    random_effects = tf.gather(
        random_effect_weights * log_county_uranium_ppm,
        indices=tf.to_int32(county),
        axis=-1)
    linear_predictor = fixed_effects + random_effects
    return tfp.distributions.Normal(
        loc=linear_predictor, scale=fwd_scale_transform(raw_likelihood_scale))


make_radon_likelihood = tf.make_template(
    name_='make_radon_likelihood', func_=_make_radon_likelihood)

In [12]:
def joint_log_prob(
    random_effect_weights, radon, floor, county,log_county_uranium_ppm, dtype
):
    num_counties = len(log_county_uranium_ppm)
    rv_weights = make_weights_prior(num_counties, dtype)
    rv_radon = make_radon_likelihood(
        random_effect_weights,
        floor,
        county,
        log_county_uranium_ppm,
        init_radon_stddev=radon.std())
    return (
        rv_weights.log_prob(random_effect_weights)+ \
        tf.reduce_sum(rv_radon.log_prob(radon), axis=-1)
    )

In [13]:
# Specify unnormalized posterior.

dtype = np.float32

log_county_uranium_ppm = radon[
    ['county', 'log_uranium_ppm']].drop_duplicates().values[:, 1]
log_county_uranium_ppm = log_county_uranium_ppm.astype(dtype)

def unnormalized_posterior_log_prob(random_effect_weights):
    return joint_log_prob(
        random_effect_weights=random_effect_weights,
        radon=dtype(radon.radon.values),
        floor=dtype(radon.floor.values),
        county=np.int32(radon.county.values),
        log_county_uranium_ppm=log_county_uranium_ppm,
        dtype=dtype
    )

In [14]:
# Set-up E-step.

step_size_hmc = tf.get_variable(
    'step_size_hmc',
    initializer=np.array(0.2, dtype=dtype),
    trainable=False)

hmc = tfp.mcmc.HamiltonianMonteCarlo(
    target_log_prob_fn=unnormalized_posterior_log_prob,
    num_leapfrog_steps=2,
    step_size=step_size_hmc,
    step_size_update_fn=tfp.mcmc.make_simple_step_size_update_policy(
        num_adaptation_steps=None),
    state_gradients_are_stopped=True)

init_random_weights = tf.placeholder(dtype, shape=[len(log_county_uranium_ppm)])

posterior_random_weights, kernel_results = tfp.mcmc.sample_chain(
    num_results=3,
    num_burnin_steps=0,
    num_steps_between_results=0,
    current_state=init_random_weights,
    kernel=hmc)

/anaconda3/envs/tfp/lib/python3.6/site-packages/numpy/core/_methods.py:122: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)
/anaconda3/envs/tfp/lib/python3.6/site-packages/numpy/core/_methods.py:122: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)


In [15]:
# Set-up M-step.

loss = -tf.reduce_mean(kernel_results.accepted_results.target_log_prob)

global_step = tf.train.get_or_create_global_step()

learning_rate = tf.train.exponential_decay(
    learning_rate=0.1,
    global_step=global_step,
    decay_steps=2,
    decay_rate=0.99)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss, global_step=global_step)

In [16]:
init_op = tf.global_variables_initializer()

In [17]:
# Grab variable handles for diagnostic purposes.

with tf.variable_scope('make_weights_prior', reuse=True):
    prior_scale = fwd_scale_transform(
        tf.get_variable(
        name='raw_prior_scale', dtype=dtype
        ))

In [18]:
with tf.variable_scope('make_radon_likelihood', reuse=True):
    likelihood_scale = fwd_scale_transform(tf.get_variable(
        name='raw_likelihood_scale', dtype=dtype))
    fixed_effect_weights = tf.get_variable(
        name='fixed_effect_weights', dtype=dtype)

In [19]:
init_op.run()
w_ = np.zeros([len(log_county_uranium_ppm)], dtype=dtype)

In [20]:
maxiter = int(15)
num_accepted = 0
num_drawn = 0
for i in range(maxiter):
    [
        _,
        global_step_,
        loss_,
        posterior_random_weights_,
        kernel_results_,
        step_size_hmc_,
        prior_scale_,
        likelihood_scale_,
        fixed_effect_weights_,
    ] = sess.run([
        train_op,
        global_step,
        loss,
        posterior_random_weights,
        kernel_results,
        step_size_hmc,
        prior_scale,
        likelihood_scale,
        fixed_effect_weights,
    ], feed_dict={init_random_weights: w_})
    w_ = posterior_random_weights_[-1, :]
    num_accepted += kernel_results_.is_accepted.sum()
    num_drawn += kernel_results_.is_accepted.size
    acceptance_rate = num_accepted / num_drawn
    if i % 100 == 0 or i == maxiter - 1:
        print(
            'global_step:{:>4}  loss:{: 9.3f}  acceptance:{:.4f}  '
            'step_size:{:.4f}  prior_scale:{:.4f}  likelihood_scale:{:.4f}  '
            'fixed_effect_weights:{}'.format(
                global_step_, loss_.mean(), acceptance_rate, step_size_hmc_,
                prior_scale_, likelihood_scale_, fixed_effect_weights_))              

global_step:   1  loss:      inf  acceptance:0.0000  step_size:0.1941  prior_scale:1.0000  likelihood_scale:inf  fixed_effect_weights:[0. 1.]
global_step:  15  loss:      nan  acceptance:0.0000  step_size:0.1278  prior_scale:nan  likelihood_scale:nan  fixed_effect_weights:[nan nan]


In [21]:
posterior_random_weights_final, kernel_results_final = tfp.mcmc.sample_chain(
    num_results=int(15e3),
    num_burnin_steps=int(1e3),
    current_state=init_random_weights,
    kernel=tfp.mcmc.HamiltonianMonteCarlo(
      target_log_prob_fn=unnormalized_posterior_log_prob,
      num_leapfrog_steps=2,
      step_size=step_size_hmc))

[
    posterior_random_weights_final_,
    kernel_results_final_,
] = sess.run([
    posterior_random_weights_final,
    kernel_results_final,
], feed_dict={init_random_weights: w_})

/anaconda3/envs/tfp/lib/python3.6/site-packages/numpy/core/_methods.py:122: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)
/anaconda3/envs/tfp/lib/python3.6/site-packages/numpy/core/_methods.py:122: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)


KeyboardInterrupt: 

In [ ]:
radon.radon.hist(bins=100)